<a href="https://colab.research.google.com/github/ShikharV010/gist_daily_runs/blob/main/Stored_Proc_Load.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages
!pip install psycopg2-binary pandas

import psycopg2
import pandas as pd
import time
from datetime import datetime

# PostgreSQL connection parameters
pg_params = {
    'host': 'gw-postgres-dev.celzx4qnlkfp.us-east-1.rds.amazonaws.com',
    'database': 'gw_prod',
    'user': 'airbyte_user',
    'password': 'airbyte_user_password',
    'port': '5432'
}

def create_audit_table():
    """Create the audit table if it doesn't exist"""
    try:
        conn = psycopg2.connect(**pg_params)
        cur = conn.cursor()

        # Create audit table if it doesn't exist
        cur.execute('''
            CREATE TABLE IF NOT EXISTS gist.audit_automated_procsandviews (
                id SERIAL PRIMARY KEY,
                object_name VARCHAR(255),
                object_type VARCHAR(50),
                start_time TIMESTAMP,
                end_time TIMESTAMP,
                duration_seconds NUMERIC(10,2),
                status VARCHAR(50),
                error_message TEXT
            )
        ''')

        conn.commit()
        print("Audit table created or already exists.")

    except Exception as e:
        print(f"Error creating audit table: {e}")
    finally:
        if cur:
            cur.close()
        if conn:
            conn.close()

def get_procedures_and_matviews():
    """Get all stored procedures and materialized views from the view"""
    try:
        conn = psycopg2.connect(**pg_params)
        cur = conn.cursor()

        # Query the gist_procsandviews view
        query = "SELECT name, object_type FROM gist.gist_procsandviews ORDER BY object_type, name"
        cur.execute(query)
        results = cur.fetchall()

        procedures = []
        matviews = []

        for name, obj_type in results:
            if obj_type == 'PROCEDURE':
                procedures.append(name)
            elif obj_type == 'MATERIALIZED VIEW':
                matviews.append(name)

        return procedures, matviews

    except Exception as e:
        print(f"Error getting objects from view: {e}")
        return [], []
    finally:
        if cur:
            cur.close()
        if conn:
            conn.close()

def execute_procedure(proc_name):
    """Execute a stored procedure and log the results"""
    conn = None
    cur = None

    # Ensure the procedure name has the schema prefix
    if not proc_name.startswith('gist.'):
        full_proc_name = f"gist.{proc_name}"
    else:
        full_proc_name = proc_name

    try:
        # Connect to the database
        conn = psycopg2.connect(**pg_params)

        # Create a separate connection for each procedure to avoid temp table conflicts
        conn.autocommit = True
        cur = conn.cursor()

        # Record start time
        start_time = datetime.now()

        # Execute the stored procedure
        print(f"Executing {full_proc_name}() at {start_time}")
        cur.execute(f"CALL {full_proc_name}()")

        # Record end time
        end_time = datetime.now()
        duration = (end_time - start_time).total_seconds()

        # Log the execution in the audit table
        cur.execute('''
            INSERT INTO gist.audit_automated_procsandviews
            (object_name, object_type, start_time, end_time, duration_seconds, status, error_message)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        ''', (full_proc_name, 'PROCEDURE', start_time, end_time, duration, 'SUCCESS', None))

        print(f"Completed {full_proc_name}() in {duration:.2f} seconds")

    except Exception as e:
        # Log error in the audit table
        error_message = str(e)
        end_time = datetime.now()
        duration = (end_time - start_time).total_seconds() if 'start_time' in locals() else 0

        if conn and not conn.closed:
            error_cur = conn.cursor()
            error_cur.execute('''
                INSERT INTO gist.audit_automated_procsandviews
                (object_name, object_type, start_time, end_time, duration_seconds, status, error_message)
                VALUES (%s, %s, %s, %s, %s, %s, %s)
            ''', (full_proc_name, 'PROCEDURE', start_time if 'start_time' in locals() else None,
                 end_time, duration, 'ERROR', error_message))
            error_cur.close()

        print(f"Error executing {full_proc_name}(): {error_message}")

    finally:
        # Close database connection
        if cur:
            cur.close()
        if conn:
            conn.close()

def refresh_matview(matview_name):
    """Refresh a materialized view and log the results"""
    conn = None
    cur = None

    # Ensure the matview name has the schema prefix
    if not matview_name.startswith('gist.'):
        full_matview_name = f"gist.{matview_name}"
    else:
        full_matview_name = matview_name

    try:
        # Connect to the database
        conn = psycopg2.connect(**pg_params)
        conn.autocommit = True
        cur = conn.cursor()

        # Record start time
        start_time = datetime.now()

        # Refresh the materialized view
        print(f"Refreshing {full_matview_name} at {start_time}")
        cur.execute(f"REFRESH MATERIALIZED VIEW {full_matview_name}")

        # Record end time
        end_time = datetime.now()
        duration = (end_time - start_time).total_seconds()

        # Log the refresh in the audit table
        cur.execute('''
            INSERT INTO gist.audit_automated_procsandviews
            (object_name, object_type, start_time, end_time, duration_seconds, status, error_message)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        ''', (full_matview_name, 'MATERIALIZED VIEW', start_time, end_time, duration, 'SUCCESS', None))

        print(f"Refreshed {full_matview_name} in {duration:.2f} seconds")

    except Exception as e:
        # Log error in the audit table
        error_message = str(e)
        end_time = datetime.now()
        duration = (end_time - start_time).total_seconds() if 'start_time' in locals() else 0

        if conn and not conn.closed:
            error_cur = conn.cursor()
            error_cur.execute('''
                INSERT INTO gist.audit_automated_procsandviews
                (object_name, object_type, start_time, end_time, duration_seconds, status, error_message)
                VALUES (%s, %s, %s, %s, %s, %s, %s)
            ''', (full_matview_name, 'MATERIALIZED VIEW', start_time if 'start_time' in locals() else None,
                 end_time, duration, 'ERROR', error_message))
            error_cur.close()

        print(f"Error refreshing {full_matview_name}: {error_message}")

    finally:
        # Close database connection
        if cur:
            cur.close()
        if conn:
            conn.close()

def run_all_objects():
    """Run all stored procedures and refresh all materialized views"""
    print(f"Starting execution at {datetime.now()}")

    # Create the audit table first
    create_audit_table()

    # Get all procedures and materialized views
    procedures, matviews = get_procedures_and_matviews()

    print(f"Found {len(procedures)} procedures and {len(matviews)} materialized views")

    # First execute all procedures
    print("\n--- EXECUTING STORED PROCEDURES ---")
    for proc in procedures:
        execute_procedure(proc)
        # Wait 10 seconds between executions
        time.sleep(10)

    # Then refresh all materialized views
    print("\n--- REFRESHING MATERIALIZED VIEWS ---")
    for matview in matviews:
        refresh_matview(matview)
        # Wait 5 seconds between refreshes
        time.sleep(5)

    print(f"Completed all executions at {datetime.now()}")

# Run all objects when the script is executed
run_all_objects()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 13.8 MB/s eta 0:00:00
Starting execution at 2025-06-18 06:16:34.189735
Audit table created or already exists.
Found 7 procedures and 4 materialized views

--- EXECUTING STORED PROCEDURES ---
Executing gist.update_accounts_csm() at 2025-06-18 06:16:35.037665
Error executing gist.update_accounts_csm(): duplicate key value violates unique constraint "gist_accountscsm_pkey"
DETAIL:  Key (client_domain)=(southeastclientservicesinc.com) already exists.
CONTEXT:  SQL statement "INSERT INTO gist.gist_accountscsm (
            client_domain, clean_client_domain, status, csm, 
            "created_at", "last_updated"
        )
        SELECT 
            client_domain, clean_client_domain, status, csm,
            v_timestamp, v_timestamp
        FROM temp_records
        WHERE record_type = 'INSERT'"
PL/pgSQL function gist.update_accounts_csm() line 167 at SQL statement

Executing gist.update_accounts_master() at 2025-06-18 06:16:45.408778
